In [41]:
%matplotlib inline

import sys
sys.path.append('../lib')

*We spoke about the basic operations like **binding** and **bundling**, but the **sub-symbolic** level allows us to do other operations which are "composite" and have no equivalent in non VSA architecture, that is because we can use SDP Algebra as I mentioned.*


## Analogy mapping

Humans use analogy all the time, it is integral part of intelligence. What if I told you we can mimic a proto form of analogy on syb-symbolic level via vector math. Nooo waaay ! .... way!

Here is the idea :

$ analogy\_map = relation * reverse\_relation $

if we do the algebraic manipulation :

$ analogy\_map * relation = reverse\_relation $

because the **bind** operation is two way street. 

Now we can use the first equation for a **training** operation and the second as **predicting/testing** operation.
The important part is that we can do this operation on a whole **structure** too, not just on single term.
Plus this is **one-shot** learning, something you can't do with Neuro-networks.

Let concretize the idea, the relations we will pick are **"above"** and correspondingly the reverse **"below"**. We will train **"analogical map"** that will virtually "swap" the position of two relations i.e. make one relation transform to the other. 
Here is what we have :

-- *circle above a square* --

$ cas = above + a\_role1 * circle + a\_role2 * square $


Let see the reverse :

-- *square below a circle* --

$ sbc = below + b\_role1 * square + b\_role2 * circle $


Now we learn the mapping (one-shot learning) :

$ map = cas * sbc $

and then we can apply it to unknown (not trained with) objects. Lets define them, so that we can do the comparison.


$ sat = above + a\_role1 * star + a\_role2 * triangle $

$ tbs = below + b\_role1 * triangle + b\_role2 * star $

So if we want to transform the "triangle" above the "star' =to=> 'star' below 'triangle', we will bind it with the learned map. Of course it is approximate match. That is vectors for you.

`tbs <=~=> map * sat`

`sat <=~=> map * tbs`

The operation works both ways. 

If you want it to work only in one direction you should use permutation-bind when defining (have not tested it).

These kind of operations are also called **holistic transformations** where one compositional structure is mapped onto another compositional structure without having to first decompose the source representation into its components

Ok now that we know the theory, let's try it.

In [42]:
from bi import *
from lexicon import *

x = lex()
x.add_items(['above', 'below', 'a1', 'a2', 'b1', 'b2', 'circle', 'square', 'star', 'triangle'])

We should use sdp.bundle(), instead of **+**, because of the even-oddity I mentioned earlier /adds too much noise and it may not work/.

In [43]:
#training data
cas = sdp.bundle([ x['above'], x['a1'] * x['circle'],   x['a2'] * x['square'] ])
sbc = sdp.bundle([ x['below'], x['b1'] * x['square'],   x['b2'] * x['circle'] ])

#novel/testing data
sat = sdp.bundle([ x['above'], x['a1'] * x['star'],     x['a2'] * x['triangle'] ])
tbs = sdp.bundle([ x['below'], x['b1'] * x['triangle'], x['b2'] * x['star'] ])

#partially closer to the training data : 'square' is used in both at the same position
sas = sdp.bundle([ x['above'], x['a1'] * x['star'],     x['a2'] * x['square'] ])
sbs = sdp.bundle([ x['below'], x['b1'] * x['square'],   x['b2'] * x['star'] ])

#misplased 'square' i.e. novel data
sas2 = sdp.bundle([ x['above'], x['a1'] * x['square'],     x['a2'] * x['star'] ])
sbs2 = sdp.bundle([ x['below'], x['b1'] * x['star'],   x['b2'] * x['square'] ])

Lets learn the map (one-shot learning) :

In [44]:
M = cas * sbc

now test it against the training example (measuring distance, btw). Seems OK.

In [45]:
(M * cas) % sbc

0

Now lets try against the test structure, which we did not use in training : 

In [46]:
(M * sat) % tbs

3720

In our case as we mentioned the mapping is bi-directional, so :

In [47]:
(M * tbs) % sat

3720

The same using sdp.dist()

In [48]:
sdp.dist((M * sat), tbs)

3720

The distance is ~38%, but we said that a distance below ~42% means that the symbols are similar (sdp.true_thresh).

In [49]:
sdp.sim((M * sat), tbs)

True

Lets also test it with data that partially match with the training data :

In [50]:
sdp.dist((M * sas), sbs)

2450

The distance as expected is smaller ~25%. 

Once more but this time 'square' is placed in the 'wrong' position i.e. it will represent again in a sense novell data, ergo distance again becomes ~37%.

In [51]:
sdp.dist((M * sas2), sbs2)

3788

And finally lets test against random SDP, to see if they are orthogonal.

In [52]:
sdp.dist((M * sat), sdp.rand())

4999

That ends our exploration of analogy mapping, but I suppose you can guess that there is big unexplored territory in the sub-symbolic space for other ways of building composite-operations on structures, if you follow the ideas described in this article. 

You can see more detailed test at **'test/analogy_mapping.py'**.

----

## Categorization

Another common human trait is **Categorization** or said it otherwise **Concept creation**.
Here again **sub-symbolic** operations can help us.
Lets try very rudimentary categorization.

We have several instances of chairs and we want to crystallize from them the concept of 'chairs'. (BTW we can ground those symbols on NN-Labels as we did with the 'summing' example).

In [53]:
x.add_items(['wooden_chair', 'metal_chair', 'special_chair', 'wooden_table', 'chair'])

So we want to be able to classify those chairs, by default as we know all the symbols are orthogonal. We need a way to make them more similar, for this reason we need an anchor that will serve the purpose of the **category** of **chairs**.

In [54]:
#chairs are orthogonal to the concept
print x['wooden_chair'] % x['chair']
print x['metal_chair'] % x['chair']

print x['wooden_chair'] % x['metal_chair']
print sdp.sim(x['wooden_chair'], x['metal_chair'])

4987
4918
4927
False


We know that the **bundle** operation aside from bundling also result in a vector closer to its operands (by hamming distance). 

So if we bundle every type of chair with the 'chair'-SDP we will move the vectors towards it, making them at the same time more similar themselves. (Of course if we use lexicon as we do here we have to update them using the **.set()** method).

In [55]:
new_wc = x['wooden_chair'] + x['chair']
x.set('wooden_chair', new_wc)

new_mc = x['metal_chair'] + x['chair']
x.set('metal_chair', new_mc)

Lets compare again :

In [56]:
print x['wooden_chair'] % x['metal_chair']
print sdp.sim(x['wooden_chair'], x['metal_chair'])

3679
True


Now the distance shrunk, both against the other chairs and the chair-concept.

In [57]:
print x['wooden_chair'] % x['chair']
print x['metal_chair'] % x['chair']

2465
2458


Also the modified chairs are still orthogonal to the rest of the symbols.

In [58]:
print x['wooden_chair'] % x['wooden_table']
print x['metal_chair'] % x['wooden_table']

5085
5044


As I said this is rudimentary categorization, just to give you yet another idea of how to use the sub-symbolic  algebra. As I'm experimenting, I'm also contemplating a ways to integrate the SDP-algebra into the higher levels, such as the Prolog syntax.
